[View in Colaboratory](https://colab.research.google.com/github/Yozh2/ClocksOrCrocs/blob/master/Clock_or_Crocodile.ipynb)

# Распознавание часов и крокодилов на изображениях с помощью предварительно обученной нейронной сети VGG16

**Источник данных** - [Архив с изображениями](https://drive.google.com/file/d/1JbYmH50iRkMorFk0xNCnwC9xKiB60Mlq/view?usp=sharing), полученный по почте.

Для распознавания используется предварительно обученная сверточная нейронная сеть VGG16.


In [0]:
import os
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.applications import VGG16
from tensorflow.python.keras.optimizers import Adam


Перед использованием данных, их необходимо разбить на обучающую, проверочную и тестовую выборки. Делается это при помощи скрипта `data_preparation`

In [0]:
# Каталог с данными для обучения
train_dir = './train'
# Каталог с данными для проверки
val_dir = './val'
# Каталог с данными для тестирования
test_dir = './test'
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Размер мини-выборки
batch_size = 64
# Количество изображений для обучения
nb_train_samples = 350
# Количество изображений для проверки
nb_validation_samples = 75
# Количество изображений для тестирования
nb_test_samples = 75

Здесь немного кода, чтобы загружать данные из папки с Google Drive в виртуальную машину Google Colab, на которой производится обучение нейронной сети. На практике оказалось проще загружать архив с датасетом, а потом разархиввировать его внутри.

In [41]:
"""
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
folder_id = '1UKpoczwNIzvZaASLBWn8Z6h2Fyk7v9Tk'

def copy_directory(source_id, local_target):
    try:
        os.makedirs(local_target)
    except: 
        pass
    file_list = drive.ListFile({'q': "'{source_id}' in parents".format(source_id=source_id)}).GetList()
    for f in file_list:
        if f["title"].startswith("."):
            continue
        fname = os.path.join(local_target, f['title'])
        if f['mimeType'] == 'application/vnd.google-apps.folder':
            copy_directory(f['id'], fname)
        else:
            f_ = drive.CreateFile({'id': f['id']})
            f_.GetContentFile(fname)

copy_directory(folder_id, './data')
"""

'\n!pip install -U -q PyDrive\n\nfrom pydrive.auth import GoogleAuth\nfrom pydrive.drive import GoogleDrive\nfrom google.colab import auth\nfrom oauth2client.client import GoogleCredentials\n\n# 1. Authenticate and create the PyDrive client.\nauth.authenticate_user()\ngauth = GoogleAuth()\ngauth.credentials = GoogleCredentials.get_application_default()\ndrive = GoogleDrive(gauth)\n\n# PyDrive reference:\n# https://googledrive.github.io/PyDrive/docs/build/html/index.html\n\n# choose a local (colab) directory to store the data.\nlocal_download_path = os.path.expanduser(\'~/data\')\ntry:\n  os.makedirs(local_download_path)\nexcept: pass\n\n# 2. Auto-iterate using the query syntax\n#    https://developers.google.com/drive/v2/web/search-parameters\nfolder_id = \'1UKpoczwNIzvZaASLBWn8Z6h2Fyk7v9Tk\'\n\ndef copy_directory(source_id, local_target):\n    try:\n        os.makedirs(local_target)\n    except: \n        pass\n    file_list = drive.ListFile({\'q\': "\'{source_id}\' in parents".format

Загружаем архив с датасетом

Saving data.zip to data.zip


Разархивируем датасет в папку

total 196
-rw-r--r-- 1 root root 2745 May  1 22:28 clock.425.png
-rw-r--r-- 1 root root 2566 May  1 22:28 clock.426.png
-rw-r--r-- 1 root root 2237 May  1 22:28 clock.427.png
-rw-r--r-- 1 root root 2518 May  1 22:28 clock.430.png
-rw-r--r-- 1 root root 2708 May  1 22:28 clock.431.png
-rw-r--r-- 1 root root 1918 May  1 22:28 clock.432.png
-rw-r--r-- 1 root root 2422 May  1 22:28 clock.433.png
-rw-r--r-- 1 root root 2458 May  1 22:28 clock.434.png
-rw-r--r-- 1 root root 2280 May  1 22:28 clock.435.png
-rw-r--r-- 1 root root 2689 May  1 22:28 clock.436.png
-rw-r--r-- 1 root root 2814 May  1 22:28 clock.437.png
-rw-r--r-- 1 root root 2709 May  1 22:28 clock.438.png
-rw-r--r-- 1 root root 2795 May  1 22:28 clock.439.png
-rw-r--r-- 1 root root 2432 May  1 22:28 clock.440.png
-rw-r--r-- 1 root root 2965 May  1 22:28 clock.441.png
-rw-r--r-- 1 root root 2259 May  1 22:28 clock.442.png
-rw-r--r-- 1 root root 2150 May  1 22:28 clock.443.png
-rw-r--r-- 1 root root 2351 May  1 22:28 clock.444.png


## Загружаем предварительно обученную нейронную сеть

In [0]:
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

"Замораживаем" веса предварительно обученной нейронной сети VGG16

In [0]:
vgg16_net.trainable = False

In [8]:
vgg16_net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

## Создаем составную нейронную сеть на основе VGG16

In [0]:
model = Sequential()
# Добавляем в модель сеть VGG16 вместо слоя
model.add(vgg16_net)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total para

Компилируем составную нейронную сеть

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-5), 
              metrics=['accuracy'])

## Создаем генератор изображений

Генератор изображений создается на основе класса ImageDataGenerator. Генератор делит значения всех пикселов изображения на 255.

In [0]:
datagen = ImageDataGenerator(rescale=1. / 255)


Генератор данных для обучения на основе изображений из каталога

In [0]:
!ls 

In [14]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

OSError: ignored

Генератор данных для проверки на основе изображений из каталога

In [0]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


Found 3750 images belonging to 2 classes.


Генератор данных для тестирования на основе изображений из каталога

In [0]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 3750 images belonging to 2 classes.


## Обучаем модель с использованием генераторов

train_generator - генератор данных для обучения

validation_data - генератор данных для проверки

In [0]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/10
273/273 [==============================] - 462s - loss: 0.2524 - acc: 0.8857 - val_loss: 0.1329 - val_acc: 0.9472
Epoch 2/10
273/273 [==============================] - 458s - loss: 0.0975 - acc: 0.9630 - val_loss: 0.1237 - val_acc: 0.9512
Epoch 3/10
273/273 [==============================] - 454s - loss: 0.0606 - acc: 0.9762 - val_loss: 0.0804 - val_acc: 0.9710
Epoch 4/10
273/273 [==============================] - 454s - loss: 0.0446 - acc: 0.9840 - val_loss: 0.0791 - val_acc: 0.9696
Epoch 5/10
273/273 [==============================] - 454s - loss: 0.0254 - acc: 0.9915 - val_loss: 0.0976 - val_acc: 0.9699
Epoch 6/10
273/273 [==============================] - 453s - loss: 0.0169 - acc: 0.9942 - val_loss: 0.0883 - val_acc: 0.9693
Epoch 7/10
273/273 [==============================] - 451s - loss: 0.0125 - acc: 0.9960 - val_loss: 0.0702 - val_acc: 0.9740
Epoch 8/10
273/273 [==============================] - 452s - loss: 0.0146 - acc: 0.9951 - val_loss: 0.1078 - val_acc: 0.9704


## Оцениваем качество работы сети с помощью генератора

In [0]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)

In [0]:
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

Аккуратность на тестовых данных: 97.31%


# Тонкая настройка сети (fine tuning)

"Размораживаем" последний сверточный блок сети VGG16

In [0]:
vgg16_net.trainable = True
trainable = False
for layer in vgg16_net.layers:
    if layer.name == 'block5_conv1':
        trainable = True
    layer.trainable = trainable    

In [0]:
# Проверяем количество обучаемых параметров
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total para

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-5), 
              metrics=['accuracy'])

In [0]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=2,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/2
273/273 [==============================] - 184s - loss: 3.6300e-04 - acc: 0.9999 - val_loss: 0.1374 - val_acc: 0.9734
Epoch 2/2
273/273 [==============================] - 184s - loss: 4.0974e-04 - acc: 0.9999 - val_loss: 0.1476 - val_acc: 0.9759


In [0]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

Аккуратность на тестовых данных: 97.15%
